In [20]:
from pprint import pprint

In [1]:
import copy

import requests
from bs4 import BeautifulSoup
from langchain.document_loaders import AsyncHtmlLoader, WebBaseLoader
from langchain.document_transformers import BeautifulSoupTransformer
from lxml import html

TEST_URLS = {
    "goldman": {
        "url": "https://www.goldmansachs.com/intelligence/pages/the-generative-world-order-ai-geopolitics-and-power.html",
        "xpath": "//div[contains(@class='article-content-page__content')]",
    },
}
# url = "https://mindmatters.ai/2023/12/large-language-models-are-still-smoke-and-mirrors/"


def web_loader(url: str, xpath: str):
    loader = AsyncHtmlLoader(url)
    raw_pages = loader.load()
    print(len(raw_pages[0].page_content))

    # my transform
    my_transformed_pages = copy.deepcopy(raw_pages)
    # extracted_text = soup.find('main').text.replace('\n\n', '')
    for page in my_transformed_pages:
        soup = BeautifulSoup(page.page_content)
        # extracted_text = str(soup.find("main"))  # for smoke and mirrors
        extracted_text = str(
            soup.find("div", attrs={"class": "article-content-page__content"})
        )  # for George's article
        # extracted_text = str(html.fromstring(str(soup)).xpath(xpath))  # for George's article
        page.page_content = extracted_text
    return my_transformed_pages


# docs = web_loader(**TEST_URLS['goldman'])
# assert len(docs) == 1
# print(docs[0].page_content[:2000])

In [13]:
# from langchain.document_loaders import OnlinePDFLoader, UnstructuredPDFLoader

# loader = UnstructuredPDFLoader(
#     "data_pdfs/tepco-tfcd-2023.pdf",
#     # mode="elements",
# )
# data = loader.load()

In [2]:
# https://github.com/nlmatics/llmsherpa?tab=readme-ov-file#read-a-pdf-file
# from llmsherpa.readers import LayoutPDFReader

# llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
# pdf_url = "data_pdfs/tepco-tfcd-2023.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
# pdf_reader = LayoutPDFReader(llmsherpa_api_url)
# doc = pdf_reader.read_pdf(pdf_url)

In [6]:
# for chunk in doc.chunks():
#     print(chunk.to_context_text())
#     print("---")

安心・安全な カーボンニュートラル社会への貢献
カーボンニュートラルは「安心で快適なくらし」に、防災は「安全で持続可能な社会」に必要不可欠です。 TEPCOグループは、供給（電力のゼロエミッション化）、系統（大規模電源・大量送電と地産地消型電源の併存）、社 会（エネルギー需要の電化）の3つの分野ごとのカーボンニュートラルロードマップを策定し、社会とともに持続可能な 成長を実現してまいります。
---
安心・安全な カーボンニュートラル社会への貢献
指標・目標戦略
---
安心・安全な カーボンニュートラル社会への貢献 > 目標
2030年度 販売電力由来のCO2排出量 50%削減（2013年度比）※2 ※2 Scope1,2,3の販売電力由来。Scope1,2は2019年度比 目標の定義はP95 2050年 エネルギー供給由来のCO2排出 実質ゼロ ロードマップ
---
安心・安全な カーボンニュートラル社会への貢献 > 目標 > 供 給
2030 2050 系 統
---
カーボン ハーフ
投資※1 事業活動
---
カーボン ハーフ
3兆円 3兆円
---
カーボン ハーフ
再生可能エネルギーの主力電源化 P24 3兆円  分散化  原子力発電の活用 P52  「JERAゼロエミッション2050」の支援 P25 進捗
---
カーボン ハーフ
販売電力由来のCO2排出量（万t-CO2）
---
カーボン ハーフ
 | 2013年度 | 2022年度 | 2030年度
 | --- | --- | ---
 | 13,920 | 6,510※3 （△53%) | 6,960 （△50%）

---
カーボン ハーフ
※3 速報値。電源調達の工夫等に加え、法規制対応において、非化石証書 の調達量が増加した影響により約53%削減(目標達成) 社 会
系統利用の最適化 P26
広域化
---
カーボン ハーフ
電化促進 P27  CO2ゼロメニュー拡大 P27  まちづくり・蓄電池 P28  EV・充電ネットワーク普及拡大 P31
---
カーボン ハーフ > カーボン ニュートラル 社会の実現
戦略 「カーボンニュートラル」と「防災」を軸とした価 値創造に向け、非化石電源である原子力等を活用 し、水力や洋上風力等の再生可能エネルギーの 開発に取り組むとともに、電気(k

In [2]:
from langchain.text_splitter import HTMLHeaderTextSplitter

def load_and_split(url, xpath):
    headers_to_split_on = [
        ("h1", "Header 1"),
        ("h2", "Header 2"),
        ("h3", "Header 3"),
    ]
    my_transformed_pages = web_loader(url, xpath)
    html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
    html_header_splits = html_splitter.split_text(my_transformed_pages[0].page_content)
    return html_header_splits

# split_docs = load_and_split(**test_urls['goldman'])
# assert len(split_docs) == 25, split_docs
# pprint([h.metadata for h in split_docs])